# Day 01 - Tutorial 01 - Elbow 2D

This series of [OpenFOAM](https://www.openfoam.com) tutorials is intended to be an extension of the classical [3-week series](https://wiki.openfoam.com/index.php?title=%223_weeks%22_series) but it is not limited to those cases. You are expected to have minimal *NIX* skills for dealing with commands in a Linux terminal and have basic knowledge of Python programming. For later tutorials some C++ knowledge is welcomed. Supported version is currently *v2106* of ESI extended version of OpenFOAM.

A different approach to the management of OpenFOAM cases is provided here. Instead of creating cases as structured directories as expected by OpenFOAM, we reunite all the files in a Jupyter Notebook and we make use of Python for generating the project structure. This is intended to make navigation easier (recent versions of JupyterLab have a good *Table of Contents* navigation) and allow for easily generating experimental cases varying parameters. During the conception of these tutorials a Python package `foamutils` is developped to cover the required data manipulations. Please notice that there is already a [`PyFoam`](https://openfoamwiki.net/index.php/Contrib/PyFoam) package for manipulating OpenFOAM data but it does not satify us as a *pythonic* package in terms of documentation and style.

With this series we propose to *sysadmins* to consider hosting [JupyterHub](https://jupyterhub.readthedocs.io/en/stable/) instances in host machines allowing access to OpenFOAM clusters/HPC for users to launch calculations based on our approach and provide VNC access for later visualization.

Since `foamutils` is in active development while these tutorials are written, `autoreload` is enabled before the package is imported. This and other details regarding the import phase early in each tutorial will not be mentioned again in later tutorials. We also preconise defining the case name early in the notebook so that whenever we want to write a file this name is available for the root directory. Needless to say given this comments, basic OpenFOAM usage will be covered less and less as tutorials get more advanced, so following them in order is a requirement for beginners.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cfdtoolbox as cfd
import cfdtoolbox.foam as fu

This tutorial extending [this elbow 2D tutorial](https://wiki.openfoam.com/index.php?title=First_simulation_by_Jozsef_Nagy) is complemented by the following materials:
- [How to run your first simulation in OpenFOAM® - Part 1 - tutorial (József Nagy)](https://www.youtube.com/watch?v=KznljrgWSvo)
- [How to run your first simulation in OpenFOAM® - Part 2 - tutorial (József Nagy)](https://www.youtube.com/watch?v=bJTSKaxG58Y)
- [PDF version of elbow 2D tutorial](https://www.cfd.at/downloads/FoamTutorial_1-ExampleOne.pdf)

In [3]:
case = "cases/foam-day-01-tutorial-01-elbow-2d"

## Basics of OpenFOAM

As any CFD or other physics-based simulation, a project in OpenFOAM requires the following steps:
1. Pre-processing:
    - Conception of geometry and a grid, this might involve multiple regions definition.
    - Select thermophysical properties for the fluids/media being simulated, including possibly chemical kinetics.
2. Modeling:
    - Select the governing equation representing the phenomena to be simulated and an adapted solver.
    - Provide adequate boundary conditions for the limits of the field and initialize internal state.
    - Select discretization schemes for the equation (notice this is not the same as meshing) and solution scheme.
    - Solve the problem or integrate in time and space and check convergence.
3. Post-processing:
    - Select regions of interest to provide visualization of results.
    - Plot contours, streamlines, vectors, etc, as applicable/required.
    - Compute other quantities from the fields.

A particularity of OpenFOAM is the way these steps are organized and how the files are stored. Although you might conceive your own solver (as we will study later in this series) that does not respect strictly what is present next, you are highly discouraged to do so, otherwise it will look weird and difficult to learn for other OpenFOAM users. An OpenFOAM case is composed, at least, of the following directories:
- `0/` (or zeros expressed in any precision you like, we discuss about that later eventually): this directory contains the boundary conditions and initial states for the patches defined in the mesh. Each physical field (pressure, velocity, temperature...) is defined in a sort of *dictionary* file, with one file per field. The required files vary from solver to solver depending on the modelled physics, but generally the naming convention is homogeneous so that velocity is `U` and pressure is `p`, for instance.
- `constant/`: this directory hosts the generated mesh in a sub-directory `polyMesh` and other files corresponding to thermophysical properties of the fluids (transport properties, heat capacities, chemical kinetics,...), definitions regarding the turbulence modelling (but not the boundary conditions, which are found under `0/`), and other problem-specific files. One important file that might appear in some cases is `fvOptions`, through which we might provide heat sources, moving reference frames, etc.
- `system/`: here go the solution control `controlDict` which is the entry point to the case, the selection of numerical schemes `fvSchemes` and the solver controls and convergence criteria in `fvSolution`. If geometry/meshing is done with OpenFOAM built-in tools, related files can also be found here. In multi-region simulations we might find sub-directories with `fvSchemes` and `fvSolutions` for different regions of the domain. Other files that might be here relate initialization of multi-phase simulations, partitioning for parallel computing, among others.

In the present tutorial we are more interested in running a first simulation, so aspects of geometry and meshing will not be covered here in depth.

## Problem geometry

The mesh is provided in `${FOAM_TUTORIALS}/resources/geometry/elbow.msh` and will later be copied to the case to convert to OpenFOAM format. In this case a `.msh` file generated with Gambit &reg; is used. The domain is composed of a large elbow with a main inlet `velocity-inlet-5` on the left side and a small channel inlet `velocity-inlet-6` coming from the bottom part. On the top we see the system outlet `pressure-outlet-7`. The other regions are treated as walls (main duct and inlet channel). A coarse mesh is used for a quick illustrative simulation. An important feature of OpenFOAM meshes that must be emphasized here is that the front and back planes `frontAndBackPlanes` in a 2D simulation as this must be separated by at least one cell over the domain thickness. In OpenFOAM a 0D simulation (`chemFoam`) is performed in a single 3D cell, 1D simulations are actually a stack of 3D cells, 2D meshes are actually extruded planes... and nothing needs to be said about 3D meshes. We will come back to this point later when we actually perform the meshing of another case.

<center>
    <img src="media/day-01-tutorial-01-elbow-2d-mesh.png" width="400px" />
</center>

## Initial conditions

In this study we make use of `icoFoam`, a solver for solution of transient incompressible laminar flows of Newtonian fluids. A complete list of standard OpenFOAM solvers can be found [here](https://www.openfoam.com/documentation/user-guide/a-reference/a.1-standard-solvers#x32-135000A.1). In this case we need only the pressure `p` and velocity `U` boundary conditions, as discussed next.

### Pressure conditions

Before entering the details of file contents we must say that all files in OpenFOAM start with a standard banned and a `FoamFile` describing what type of data it contains, the *object* it models, and some metadata. In the case of pressure we should see something as follows. Notice the `class volScalarField`, which means that we as handling a volumetric scalar quantity.

In [4]:
print(fu.make_header("p", None))

/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2106                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    object      p;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //



Since this is an incompressible solver, pressure is actually represented by the reduced pressure (pressure divided by density), thus its units are not in pascal, but $(m/s)^2$, as well as viscosity, as we see later, is provided by kinematic viscosity. With this we introduce you to the first element inside the body of any OpenFOAM conditions file, the `dimensions`, which is documented [here](https://www.openfoam.com/documentation/user-guide/2-openfoam-cases/2-2-basic-inputoutput-file-format). It is an array with the exponents used for each of the basic SI units. From the pressure units provided before, we guess that the second element represent length $L$ in meters and the third time $T$ in second seconds. The others will be presented later or you can read the link with the documentation.

The second element of this file is the `internalField`, which provides the initial condition of the quantity in the volume. Here we declare the pressure to be `uniform` and equal to zero. It can also be `nonuniform` and provided for each cell in the mesh individually. That means that you can re-use solutions of simpler problems (for instance, approximate the velocity/pressure field in steady state) to initialize more complex problems, such as multiphase or enabling chemistry. This will be discussed in a more advanced case.

Finally we declare the `boundaryField`, which as it name says, provides the boundary conditions. All patches in de domain must be provided. In the case of pressure a common wall condition is `zeroGradient` meaning what its name say... and in the outlet `pressure-outlet-7` we provide a constant `fixedValue` a and `uniform` zero value reference pressure. Notice that when specifying inlets, it is common and physical to provide boundary conditions as zero gradient, meaning that the solver will find a compatible pressure with the imposed velocities.

The fact that the case is 2D is imposed by the `empty` type applied to `frontAndBackPlanes`, what will remain the same in any other file.

The last point to keep about syntax is that any dictionary entry must be closed by a semi-colon `;` and files are nested with braces in a `C`-looking style.

In [5]:
body_p = """\
dimensions      [0 2 -2 0 0 0 0];

internalField   uniform 0;

boundaryField
{
    wall-4
    {
        type            zeroGradient;
    }

    velocity-inlet-5
    {
        type            zeroGradient;
    }

    velocity-inlet-6
    {
        type            zeroGradient;
    }

    pressure-outlet-7
    {
        type            fixedValue;
        value           uniform 0;
    }

    wall-8
    {
        type            zeroGradient;
    }

    frontAndBackPlanes
    {
        type            empty;
    }
}
"""

### Velocity conditions

For velocity `U` file we observe the change of class to `volVectorField`, as one should expect since velocity is fully described by its components.

In [6]:
print(fu.make_header("U", None))

/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2106                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       volVectorField;
    object      U;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //



The major progress from Euler's equation to Navier-Stokes' solved by `icoFoam` (and the vast majority of OpenFOAM solvers) is the treatment of viscosity, which in turn imply `noSlip` boundary conditions over walls. In the case of `internalField` and the velocity inlets notice that we declare the vector entries within parenthesis as $(V_x\;V_y\;V_z)$. Pressure outlet is modeled by a zero gradient velocity boundary condition. For now this is all regarding conditions to intialize the problem, more complex boundary conditions and non-constant profiles being discussed in the up-coming tutorials.

In [7]:
body_U = """\
dimensions      [0 1 -1 0 0 0 0];

internalField   uniform (0 0 0);

boundaryField
{
    wall-4
    {
        type            noSlip;
    }

    velocity-inlet-5
    {
        type            fixedValue;
        value           uniform (1 0 0);
    }

    velocity-inlet-6
    {
        type            fixedValue;
        value           uniform (0 3 0);
    }

    pressure-outlet-7
    {
        type            zeroGradient;
    }

    wall-8
    {
        type            noSlip;
    }

    frontAndBackPlanes
    {
        type            empty;
    }
}
"""

## Constant files

This being a very simple and laminar case, the only file required under `constant/` other then the mesh is `transportProperties`, which is of class `dictionary`. As it has been stated, here we make use of kinematic viscosity `nu` ($\nu$) with units in SI. The current version of OpenFOAM no longer require units for this (although we disagree with this poor design choice), previously you had a `[ 0 2 -1 0 0 0 0 ]` in front of the value. To provide this quantity we simply declare its keyword and value, as provided below.

In [8]:
print(fu.make_header("transportProperties", None))

/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2106                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      transportProperties;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //



In [9]:
body_transportProperties = "nu              0.01;"

## System files

A last group of files that an OpenFOAM simulation requires is those located under `system/`. These files are related to solution control, solver methods, convergence control, parallel decomposition, meshing, etc. Here we describe the files present in our case, more files will be added in the upcoming tutorials as applicable.

### Control dictionary

The `controlDict` dictionary deals with time data and input/output control and is documented [here](https://www.openfoam.com/documentation/user-guide/6-solving/6.1-time-and-data-inputoutput-control#x22-830006.1). It allows you to restart simulations from latest time if you had to stop running for some reason and eventually decrease time-step `deltaT` if convergence is not doing fine (requires `runTimeModifiable` set to `true` so the file is reinterpreted during a calculation). We can also link functions for post-processing *function objects* and other functionalities here.

**TIP:** you can add comments to OpenFOAM files with `//` or add block comments with `/* comment */` as in C/C++. This is the file that I personally make use of comments quite often to keep track of changes I do in runtime or to remmember the meaning of some less used options. You can also keep not of available options, *e.g.* as a reminder that `writeFormat` can be `ascii` for plain text results or `binary`.

In [10]:
body_controlDict = """\
application       icoFoam;

startFrom         latestTime;

startTime         0;

stopAt            endTime;

endTime           100;

deltaT            0.05;

writeControl      timeStep;

writeInterval     200;

purgeWrite        0;

writeFormat       ascii;

writePrecision    6;

writeCompression  off;

timeFormat        general;

timePrecision     6;

runTimeModifiable true;
"""

### Finite volume schemes

Numerical schemes are provided in `fvSchemes` dictionary and all available options are provided [here](https://www.openfoam.com/documentation/user-guide/6-solving/6.2-numerical-schemes). The names of the sub-dictionaries are probably self explanatory, otherwise you can check the recommended documentation. It is considered a good practice to provide `default none;` where applicable so you explicitly provide all schemes being used in the solution. Some problems become unstable under a poor selection of numerical schemes and often domain knowledge is required to provide a proper configuration. In this case we follow the sample tutorial provided with OpenFOAM leaving more details for later.

In [11]:
body_fvSchemes = """\
ddtSchemes
{
    default         Euler;
}

gradSchemes
{
    default         Gauss linear;
}

divSchemes
{
    default         none;
    div(phi,U)      Gauss limitedLinearV 1;
}

laplacianSchemes
{
    default         Gauss linear corrected;
}

interpolationSchemes
{
    default         linear;
}

snGradSchemes
{
    default         corrected;
}
"""

### Finite volume solution

The last common file in this tutorial is `fvSolution`, responsible by convergence and solver control. Its documentation is provided [here](https://www.openfoam.com/documentation/user-guide/6-solving/6.3-solution-and-algorithm-control). Notice in `solvers` sub-dictionary the presence of the variables we are solving in the problem and associated solvers/preconditioners linked to them. Another sub-dictionary `PISO` provides the control for the underlining algorithm used by `icoFoam` to integrate the problem. Depending on the solver we might have `relaxationFactors` and `SIMPLE` parameters in this file.

In [12]:
body_fvSolution = """\
solvers
{
    p
    {
        solver          PCG;
        preconditioner  DIC;
        tolerance       1e-06;
        relTol          0.05;
    }

    pFinal
    {
        $p;
        relTol          0;
    }

    U
    {
        solver          smoothSolver;
        smoother        symGaussSeidel;
        tolerance       1e-05;
        relTol          0;
    }
}

PISO
{
    nCorrectors     2;
    nNonOrthogonalCorrectors 2;
}
"""

### Fluent mesh conversion

This dictionary is specific to Fluent file conversion, we propose you look-up the documentation to get used with OpenFOAM by yourself.

In [13]:
body_foamDataToFluentDict = """\
p               1;

U               2;

T               3;

h               4;

k               5;

epsilon         6;

alpha1          150;
"""

## Write case files

This is an extremely simple simulation case so we have decided to write all the files here in the end. This was done to remind you once again about the minimum set of directories and where each configuration file must go. Adding files to the `case` is done through `fu.make_file`, which will take at least the case path, directory in the case, file name, and the string with case data as arguments. More complex uses of this function will appear later. So dump our case to file and we are prepared to run it.

In [14]:
fu.make_file(case, "0",        "p",                     body_p)
fu.make_file(case, "0",        "U",                     body_U)
fu.make_file(case, "constant", "transportProperties",   body_transportProperties)
fu.make_file(case, "system",   "controlDict",           body_controlDict)
fu.make_file(case, "system",   "fvSchemes",             body_fvSchemes)
fu.make_file(case, "system",   "fvSolution",            body_fvSolution)
fu.make_file(case, "system",   "foamDataToFluentDict",  body_foamDataToFluentDict)

## Run simulation

So far we have only discussed about the configurations for the case, but not about how to *actually* use OpenFOAM. Now it is time to do the dirty work. We did not provide a mesh or `blockMeshDict` because this case is based on an external mesh file. We copy this from the `FOAM_TUTORIALS` directory and perform the conversion to a format OpenFOAM solvers are capable to interpret. Since the provided mesh was originally conceived to be used with Ansys Fluent &reg; we do make use of `fluentMeshToFoam`.

For running commands we can use `fu.run_cmd`, which takes the case path (the working directory), a log-file name, and the command to run as arguments.

**TIP:** most OpenFOAM programs support a `-help` flag (notice the single hyphen, sometimes Linux users get confused by this convention). Try running `fluentMeshToFoam -help` to check the available options by yourself. Do the same with `icoFoam` or any other solver and supporting tool.

In [15]:
source = "/usr/lib/openfoam/openfoam2112/etc/bashrc"

In [16]:
cfd.run_cmd(case, "log.tmp", f". {source} && cp ${{FOAM_TUTORIALS}}/resources/geometry/elbow.msh .")
cfd.run_cmd(case, "log.fluentMeshToFoam", f". {source} && fluentMeshToFoam elbow.msh")

Running from cases/foam-day-01-tutorial-01-elbow-2d
Running from cases/foam-day-01-tutorial-01-elbow-2d


Now you should have a `constant/polyMesh/` directory in the case. It is important to know about the `boundary` file within `polyMesh` since often we may need to modify patches (named regions of our boundaries) to be able to apply certain boundary conditions to it, such as symmetry, periodic, etc. With that we are ready to launch the simulation. Notice the ampersand `&` in the end of the command so that we can keep editing our notebook, *e.g* watching the `log.icoFoam` file and eventually fix controls on runtime (this case will run in seconds, you won't have time to do that).

In [17]:
cfd.run_cmd(case, "log.icoFoam", f". {source} && icoFoam")

Running from cases/foam-day-01-tutorial-01-elbow-2d


OpenFOAM also provides tools to allow exporting results to other software. You could try the following to get back to Ansys Fluent &reg;.

In [18]:
cfd.run_cmd(case, "log.foamMeshToFluent", f". {source} && foamMeshToFluent")
cfd.run_cmd(case, "log.foamDataToFluent", f". {source} && foamDataToFluent")

Running from cases/foam-day-01-tutorial-01-elbow-2d
Running from cases/foam-day-01-tutorial-01-elbow-2d


After completing you can run `paraFoam` from terminal for post-processing. For the final time of this simulation we get the following velocity magnitude profile. Did you get something similar? Don't know how to use `paraview`? Try [this video series](https://www.youtube.com/playlist?list=PLvkU6i2iQ2fpcVsqaKXJT5Wjb9_ttRLK-).

<center>
    <img src="media/day-01-tutorial-01-elbow-2d-U-final.png" width="400px" />
</center>

## Additional learning

- Check the use of `foamToVTK` to use plain `paraview` instead of `paraFoam`.
- Explore alternative meshes with different refinement for this case as provided [here](https://github.com/jnmlujnmlu/OpenFOAMTeaching/tree/master/JozsefNagy).
- Compare the solutions and start studying mesh convergence as proposed in the [original tutorial](https://www.cfd.at/downloads/FoamTutorial_1-ExampleOne.pdf).

Hope you have enjoyed and keep watching for more tutorials!